<a href="https://colab.research.google.com/github/nicolemichaud03/Recipe-Recommender-System/blob/main/NNnotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neural Network Recipe Recommendation System with Embeddings for Dietary Restriction

### Pre-Processing and Data Exploration:

Loading the data and necessary packages:

In [1]:
# TF's recommender imports
!pip install -q tensorflow-recommenders
!pip install -q tensorflow_ranking
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scann 1.2.10 requires tensorflow~=2.13.0, but you have tensorflow 2.15.0.post1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-serving-api 2.14.1 requires tensorflow<3,>=2.14.1, but you have tensorflow 2.13.1 which is incompatible.


In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.1


In [3]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
import io
from collections import defaultdict
import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import tensorflow_ranking as tfr

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading the user and item data and viewing it:

In [5]:
user_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone_data/RAW_interactions.csv')
recipe_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/capstone_data/RAW_recipes.csv')

In [6]:
user_data.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [7]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132198 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [8]:

recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [9]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [10]:
recipe_data = recipe_data.rename(columns={"id": "recipe_id"})

In [11]:
print(len(user_data['user_id'].unique()))
print(len(recipe_data['recipe_id'].unique()))

226570
231637


With 226,570 users and 231,637 recipes, there are less users than there are recipes. Therefore, it is probably best for our recommender system to be user-user based.

Dropping the columns I know I won't be working with:

In [12]:
user_recipe_ratings = user_data.drop(columns=['review', 'date'])

In [13]:
recipe_data = recipe_data.drop(columns=['contributor_id', 'submitted', 'nutrition', 'steps', 'minutes', 'n_steps', 'n_ingredients'])
recipe_data = recipe_data.rename(columns={"id": "recipe_id"})

In [14]:
# Converting id values to strings to be compatible for
# later use with tensorflow
user_data['recipe_id'] = user_data['recipe_id'].astype(str)
recipe_data['recipe_id'] = recipe_data['recipe_id'].astype(str)
user_data['user_id'] = user_data['user_id'].astype(str)

In [15]:
# Making sure text features are strings so that they can be
# cleaned appropriately
recipe_data['name'] = recipe_data['name'].astype(str)
recipe_data['description'] = recipe_data['description'].astype(str)
recipe_data['tags'] = recipe_data['tags'].astype(str)

In [16]:
# Creating a function to perform cleaning steps at once
# (Removes numbers and unnecessary characters,
# makes all letters lowercase, removes stopwords)
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    text = no_nums.sub('', text)
    text = no_bad_chars.sub(' ', text)
    text = text.lower()
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
#Applying text cleaning function to text columns
recipe_data_cleaned = recipe_data.copy()
recipe_data_cleaned['name'] = (recipe_data['name']).apply(clean_text)
recipe_data_cleaned['description'] = (recipe_data['description']).apply(clean_text)
recipe_data_cleaned['tags'] = (recipe_data['tags']).apply(clean_text)
recipe_data_cleaned.head()

,name,recipe_id,tags,description,ingredients
0,arriba baked winter squash mexican style,137739,'minutesorless' 'timetomake' 'course' 'maining...,autumn favorite time year cook recipe prepared...,"['winter squash', 'mexican seasoning', 'mixed ..."
1,bit different breakfast pizza,31490,'minutesorless' 'timetomake' 'course' 'maining...,recipe calls crust prebaked bit adding ingredi...,"['prepared pizza crust', 'sausage patty', 'egg..."
2,kitchen chili,112140,'timetomake' 'course' 'preparation' 'maindish'...,modified version 'mom's' chili hit christmas p...,"['ground beef', 'yellow onions', 'diced tomato..."
3,alouette potatoes,59389,'minutesorless' 'timetomake' 'course' 'maining...,super easy great tasting make ahead side dish ...,"['spreadable cheese with garlic and herbs', 'n..."
4,amish tomato ketchup canning,44061,'weeknight' 'timetomake' 'course' 'mainingredi...,dh's amish mother raised recipe much prefers s...,"['tomato juice', 'apple cider vinegar', 'sugar..."


Feature Engineering to categorize each recipe as different diet types (Vegetarian, Vegan, and/or Gluten-Free):

In [18]:
# creating a new column to classify recipes as gluten-free or not
GF = []
#tags column contains the most information on gluten-free recipes
#(see miscellaneous notebook)
for row in recipe_data_cleaned['tags']:
    if "gluten-free" in row : GF.append("Gluten-Free")
    elif "gluten free" in row : GF.append("Gluten-Free")
    else: GF.append("None")

In [19]:
recipe_data_cleaned['GF'] = GF

In [20]:
#Ingredient lists for diet filtering:
vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']

In [21]:
# creating two new columns to classify recipes as vegetarian or not
# and as vegan or not:

recipe_data_cleaned['vegetarian'] = None
recipe_data_cleaned['vegan'] = None

In [22]:
# Filtering through the 'ingedients' column for ingredients that
# aren' t vegetarian or vegan
vege_pattern = '|'.join(vegetarian)
vegan_pattern = '|'.join(vegan)


recipe_data_cleaned.vegetarian = recipe_data_cleaned.ingredients.str.contains(vege_pattern)
recipe_data_cleaned.vegan = recipe_data_cleaned.ingredients.str.contains(vegan_pattern)

In [23]:
# Changing Boolean values to 0 and 1 for easy counting later
recipe_data_cleaned['vegetarian'] = recipe_data_cleaned['vegetarian'].astype(str)
recipe_data_cleaned['vegetarian'] = recipe_data_cleaned['vegetarian'].replace({'False': 'Vegetarian', 'True': 'None'})

In [24]:
recipe_data_cleaned['vegan'] = recipe_data_cleaned['vegan'].astype(str)
recipe_data_cleaned['vegan'] = recipe_data_cleaned['vegan'].replace({'False': 'Vegan', 'True': 'None'})

In [25]:
#Making column names match and merging dfs to classify user diets based on the recipes they've used
recipe_data_cleaned = recipe_data_cleaned.rename(columns = {'id': 'recipe_id'})

#user_diets = pd.merge(user_data, recipe_data_cleaned, on='recipe_id', how='left')

In [26]:
#user_diets.head()

In [27]:
recipe_data_cleaned['diets_combined'] = recipe_data_cleaned[['vegetarian', 'vegan', 'GF']].values.tolist()
recipe_data_cleaned = recipe_data_cleaned.drop(columns=['GF', 'vegetarian', 'vegan'])
recipe_data_cleaned.head()

,name,recipe_id,tags,description,ingredients,diets_combined
0,arriba baked winter squash mexican style,137739,'minutesorless' 'timetomake' 'course' 'maining...,autumn favorite time year cook recipe prepared...,"['winter squash', 'mexican seasoning', 'mixed ...","[Vegetarian, None, None]"
1,bit different breakfast pizza,31490,'minutesorless' 'timetomake' 'course' 'maining...,recipe calls crust prebaked bit adding ingredi...,"['prepared pizza crust', 'sausage patty', 'egg...","[None, None, None]"
2,kitchen chili,112140,'timetomake' 'course' 'preparation' 'maindish'...,modified version 'mom's' chili hit christmas p...,"['ground beef', 'yellow onions', 'diced tomato...","[None, None, None]"
3,alouette potatoes,59389,'minutesorless' 'timetomake' 'course' 'maining...,super easy great tasting make ahead side dish ...,"['spreadable cheese with garlic and herbs', 'n...","[Vegetarian, None, None]"
4,amish tomato ketchup canning,44061,'weeknight' 'timetomake' 'course' 'mainingredi...,dh's amish mother raised recipe much prefers s...,"['tomato juice', 'apple cider vinegar', 'sugar...","[Vegetarian, Vegan, None]"


In [28]:
# #combining the diet-classified recipe data with the user data based on which recipes each user has rated

# user_diets = (user_diets.groupby(['user_id']).agg({'recipe_id': lambda x: x.tolist(), 'rating': lambda x: x.tolist(), 'GF': sum, 'vegetarian':sum, 'vegan': sum}).reset_index())
# user_diets.head()

In [29]:
# # creating new columns to classify users as the different diet types
# user_diets['is_vegetarian'] = None
# user_diets['is_vegan'] = None
# user_diets['is_GF'] = None
# user_diets.head()

In [30]:
# #getting a total count of the recipes that each user has rated
# user_diets['recipe_totals'] = user_diets['recipe_id'].str.len()

In [31]:
# #classifying a user as vegetarian if at least 75% of the recipes they've rated are vegetarian
# user_diets['is_vegetarian'] = np.where(user_diets['vegetarian'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)


In [32]:
# #classifying a user as vegan if at least 75% of the recipes they've rated are vegan
# user_diets['is_vegan'] = np.where(user_diets['vegan'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)

In [33]:
# #classifying a user as gluten-free if at least 75% of the recipes they've rated are gluten-free
# user_diets['is_GF'] = np.where(user_diets['GF'] >= ((user_diets['recipe_totals'])*(.75)), 1, 0)

### Modeling

Creating a baseline model without extra features ?

In [34]:
# user_recipe_ratings['recipe_id'] = user_recipe_ratings['recipe_id'].astype(str)
# user_recipe_ratings['user_id'] = user_recipe_ratings['user_id'].astype(str)
# #user_recipe_ratings['rating'] = user_recipe_ratings['rating'].astype(str)

In [35]:
recipe_data_cleaned['diets_combined'] = recipe_data_cleaned['diets_combined'].astype(str)

In [36]:
user_ds = tf.data.Dataset.from_tensor_slices(dict(user_recipe_ratings))

recipe_ds = tf.data.Dataset.from_tensor_slices(dict(recipe_data_cleaned))

In [37]:
#creating a multitask model
#preparing the data


# Select the basic features.
ratings = user_ds.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "user_id": x["user_id"],
    "rating": x["rating"],
})
recipes = recipe_ds.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "diet_type": x["diets_combined"]})

In [39]:
#preparations to build vocab and split data
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

recipe_ids = recipes.batch(1_000).map(lambda x: x["recipe_id"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_recipe_ids = np.unique(np.concatenate(list(recipe_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [40]:
unique_user_ids = unique_user_ids.astype(str)
unique_recipe_ids = unique_recipe_ids.astype(str)

In [37]:
# #ranking task
# tfrs.tasks.Ranking(
#     loss=tf.keras.losses.MeanSquaredError(),
#     metrics=[tf.keras.metrics.RootMeanSquaredError()],
# )

In [38]:
# #retrieval task
# tfrs.tasks.Retrieval(
#     metrics=tfrs.metrics.FactorizedTopK(
#         candidates=recipes.batch(128)
#     )
# )

In [39]:
# # since we have two tasks and two losses - we need to decide on how important each loss is
# # We can do this by giving each of the losses a weight, and treating these weights as hyperparameters


# class UserRecipesModel(tfrs.models.Model):

#   def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
#     # We take the loss weights in the constructor: this allows us to instantiate
#     # several model objects with different loss weights.

#     super().__init__()

#     embedding_dimension = 32

#     # User and recipe models.
#     self.recipe_model: tf.keras.layers.Layer = tf.keras.Sequential([
#       tf.keras.layers.StringLookup(
#         vocabulary=unique_recipe_ids, mask_token=None),
#       tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, embedding_dimension)
#     ])
#     self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
#       tf.keras.layers.StringLookup(
#         vocabulary=unique_user_ids, mask_token=None),
#       tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
#     ])

#     # A small model to take in user and recipe embeddings and predict ratings.
#     # We can make this as complicated as we want as long as we output a scalar
#     # as our prediction.
#     self.rating_model = tf.keras.Sequential([
#         tf.keras.layers.Dense(256, activation="relu"),
#         tf.keras.layers.Dense(128, activation="relu"),
#         tf.keras.layers.Dense(1),
#     ])

#     # The tasks.
#     self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
#         loss=tf.keras.losses.MeanSquaredError(),
#         metrics=[tf.keras.metrics.RootMeanSquaredError()],
#     )
#     self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
#         metrics=tfrs.metrics.FactorizedTopK(
#             candidates=recipes.batch(128).map(self.recipe_model)
#         )
#     )

#     # The loss weights.
#     self.rating_weight = rating_weight
#     self.retrieval_weight = retrieval_weight

#   def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
#     # We pick out the user features and pass them into the user model.
#     user_embeddings = self.user_model(features["user_id"])
#     # And pick out the recipe features and pass them into the recipe model.
#     recipe_embeddings = self.recipe_model(features["recipe_id"])

#     return (
#         user_embeddings,
#         recipe_embeddings,
#         # We apply the multi-layered rating model to a concatentation of
#         # user and recipe embeddings.
#         self.rating_model(
#             tf.concat([user_embeddings, recipe_embeddings], axis=1)
#         ),
#     )

#   def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

#     ratings = features.pop("rating")

#     user_embeddings, recipe_embeddings, rating_predictions = self(features)

#     # We compute the loss for each task.
#     rating_loss = self.rating_task(
#         labels=ratings,
#         predictions=rating_predictions,
#     )
#     retrieval_loss = self.retrieval_task(user_embeddings, recipe_embeddings)

#     # And combine them using the loss weights.
#     return (self.rating_weight * rating_loss
#             + self.retrieval_weight * retrieval_loss)

In [40]:
# #Ranking specialized model
# model = UserRecipesModel(rating_weight=1.0, retrieval_weight=0.0)
# model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [41]:
# #Then shuffle, batch, and cache the training and evaluation data:
# cached_train = train.shuffle(100_000).batch(8192).cache()
# cached_test = test.batch(4096).cache()

In [42]:
# model.fit(cached_train, epochs=3)
# metrics = model.evaluate(cached_test, return_dict=True)

# print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
# print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

The RMSE of this Ranking-only model was 1.267.

Retrieval top-100 accuracy: 0.001

In [43]:
# #Retrieval specialized model
# model = UserRecipesModel(rating_weight=0.0, retrieval_weight=1.0)
# model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [44]:
# model.fit(cached_train, epochs=3)
# metrics = model.evaluate(cached_test, return_dict=True)

# print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
# print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

The RMSE of this Retrieval-only model was: 4.528

Retrieval top-100 accuracy: 0.036

In [45]:
# #joint model
# model = UserRecipesModel(rating_weight=1.0, retrieval_weight=1.0)
# model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [46]:
# model.fit(cached_train, epochs=3)
# metrics = model.evaluate(cached_test, return_dict=True)

# print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
# print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

The RMSE of this combined (ranking and retrieval) model was: 1.342

Retrieval top-100 accuracy: 0.037

In [48]:
# #make predictions
# trained_recipe_embeddings, trained_user_embeddings, predicted_rating = model({
#       "user_id": np.array([" "]),
#       "recipe_id": np.array([" "])
#   })
# print("Predicted rating:")
# print(predicted_rating)

In [49]:
#recipe_ds_plus = tf.data.Dataset.from_tensor_slices(dict(recipe_data_cleaned))

Model with extra embeddings:

In [47]:
# #preparing the data
# # Select the basic features plus the additional ones for the recipes
# recipes_plus = recipe_ds.map(lambda x: {
#     "recipe_id": x["recipe_id"],
#     "vegetarian": x["vegetarian"],
#     "vegan": x["vegan"],
#     "GF": x["GF"],
#     "tags": x["tags"],
#     "description": x["description"],
#     "name": x["name"]
# })


In [48]:
# timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

# max_timestamp = timestamps.max()
# min_timestamp = timestamps.min()

# timestamp_buckets = np.linspace(
#     min_timestamp, max_timestamp, num=1000,
# )

# unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
# unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
#     lambda x: x["user_id"]))))

In [49]:
# #normalizing the diet-types columns. They are already discrete.
# timestamp_normalization = tf.keras.layers.Normalization(
#     axis=None
# )
# timestamp_normalization.adapt(ratings.map(lambda x: x["timestamp"]).batch(1024))

# for x in ratings.take(3).as_numpy_iterator():
#   print(f"Normalized timestamp: {timestamp_normalization(x['timestamp'])}.")

In [50]:
#preparations to build vocab and split data
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

#Understand this part better:

# recipe_ids = recipes_plus.batch(1_000).map(lambda x: x["recipe_id"])
# vegetarian = recipes_plus.batch(100).map(lambda x: x["vegetarian"])
# vegan = recipes_plus.batch(100).map(lambda x: x["vegan"])
# g_f = recipes_plus.batch(100).map(lambda x: x["GF"])
#user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

train_data = train.map(lambda x, y: (tf.strings.join([x["recipe_id"], x["vegetarian"], x["vegan"] x["GF"]]), y))
test_data = test.map(lambda x, y: (tf.strings.join([x["recipe_id"], x["vegetarian"], x["vegan"], x["GF"]]), y))

unique_recipe_ids = np.unique(np.concatenate(list(recipe_ids)))
unique_vegetarian_values = np.unique(np.concatenate(list(vegetarian)))
unique_vegan_values = np.unique(np.concatenate(list(vegan)))
unique_gf_values = np.unique(np.concatenate(list(g_f)))

unique_recipe_ids = unique_recipe_ids.astype(str)
unique_vegetarian_values = unique_vegetarian_values.astype(str)
unique_vegan_values = unique_vegan_values.astype(str)
unique_gf_values = unique_gf_values.astype(str)

In [ ]:
#tasks

In [41]:
#user model, extra embeddings

class UserModel(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids)+1, 32)
    ])

  def call(self, titles):
    return self.user_embedding(inputs["user_id"])


In [59]:
#recipe model, extra embeddings

class RecipeModelPlus(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.recipe_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_recipe_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_recipe_ids)+1, 32)
    ])
    # self.diets_embedding = tf.keras.Sequential([
    #     tf.keras.layers.IntegerLookup(
    #         vocabulary=unique_vegetarian_values, mask_token=None),
    #     tf.keras.layers.Embedding(len(unique_vegetarian_values) + 1 , 32)
    #  ])
    max_tokens = 10_000

    self.diet_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.diets_embedding = tf.keras.Sequential([
      self.diet_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.diet_vectorizer.adapt(recipes)

    # self.vegan_embedding = tf.keras.Sequential([
    #     tf.keras.layers.IntegerLookup(
    #         vocabulary=unique_vegan_values, mask_token=None),
    #     tf.keras.layers.Embedding(len(unique_vegan_values) + 1 , 32)
    # ])
    # self.gf_embedding = tf.keras.Sequential([
    #     tf.keras.layers.IntegerLookup(
    #         vocabulary=unique_gf_values, mask_token=None),
    #     tf.keras.layers.Embedding(len(unique_gf_values) + 1 , 32)
    # ])

  def call(self, recipes):
    return tf.concat([
        self.recipe_embedding(recipes),
        self.diets_embedding(recipes)
    ], axis=1)

    # self.user_embedding(inputs["user_id"]),
    #     self.timestamp_embedding(inputs["timestamp"]),
    #     tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    # ], axis=1)

In [60]:
class UserRecipe_plus_Model(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      RecipeModelPlus(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=recipe_ids.batch(32).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model(features["user_id"])
    recipe_embeddings = self.candidate_model({
        "recipe_id": features["recipe_id"],
        "diet_type": features["diet_type"]
    })
    return self.task(query_embeddings, recipe_embeddings)

In [61]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# train_data = train.map(lambda x, y: (tf.strings.join([x["recipe_id"], x["vegetarian"], x["vegan"], x["GF"]]), y))
# test_data = test.map(lambda x, y: (tf.strings.join([x["recipe_id"], x["vegetarian"], x["vegan"], x["GF"]]), y))


# cached_train = train_data.shuffle(1000).batch(32).cache()
# cached_test = test_data.batch(32).cache()

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [62]:

model = UserRecipe_plus_Model()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

TypeError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/base_preprocessing_layer.py", line 123, in adapt_step  *
        self.update_state(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/text_vectorization.py", line 476, in update_state  **
        self._lookup_layer.update_state(self._preprocess(data))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/text_vectorization.py", line 570, in _preprocess
        inputs = utils.ensure_tensor(inputs, dtype=tf.string)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/preprocessing_utils.py", line 32, in ensure_tensor
        inputs = tf.convert_to_tensor(inputs, dtype)

    TypeError: Expected string, but got Tensor("IteratorGetNext:0", shape=(), dtype=string) of type 'Tensor'.


In [ ]:
class UserRecipesModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and recipe models.
    self.recipe_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_recipe_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, embedding_dimension)

      vege_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_vegetarian_values, mask_token=None),
        tf.layers.experimental.Embedding(len(unique_vegetarian_values) + 1 , 32)
    ])


      ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and recipe embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=recipes.batch(128).map(self.recipe_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the recipe features and pass them into the recipe model.
    recipe_embeddings = self.recipe_model(features["recipe_id"])

    return (
        user_embeddings,
        recipe_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and recipe embeddings.
        self.rating_model(
            tf.concat([user_embeddings, recipe_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, recipe_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, recipe_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [ ]:
#Making another model but with extra embeddings (diet types and text features)
#will eventually just want to have 1 model where you can turn on or off the additional embeddings
class UserRecipesModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and recipe models.
    self.recipe_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_recipe_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, embedding_dimension)
      #add extra embeddings
      #vegetarian diet type
      tf.keras.layers.IntegerLookup(
          vocabulary = unique_vegetarian_values, mask_token=None),
      tf.keras.layers.Embedding(len(unique_vegetarian_values) + 1, embedding_dimension)
      #vegan diet type
      tf.keras.layers.IntegerLookup(
          vocabulary = unique_vegan_values, mask_token=None),
      tf.keras.layers.Embedding(len(unique_vegan_values) + 1, embedding_dimension)
      # gluten-free diet type
      tf.keras.layers.IntegerLookup(
          vocabulary = unique_gf_values, mask_token=None),
      tf.keras.layers.Embedding(len(unique_gf_values) + 1, embedding_dimension)



    ])

    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and recipe embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=recipes.batch(128).map(self.recipe_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the recipe features and pass them into the recipe model.
    recipe_embeddings = self.recipe_model(features["recipe_id"])

    return (
        user_embeddings,
        recipe_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and recipe embeddings.
        self.rating_model(
            tf.concat([user_embeddings, recipe_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("rating")

    user_embeddings, recipe_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, recipe_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [ ]:
#Query model
class UserModel(tf.keras.Model):

  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
      ])
      self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
      )

      self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    ], axis=1)

In [ ]:
#candidate model w/extra features
class RecipeModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(movies)





#add extra embeddings
      #vegetarian diet type
      self.vege_embedding = tf.keras.Sequential([
      tf.keras.layers.IntegerLookup(
          vocabulary = unique_vegetarian_values, mask_token=None),
      tf.keras.layers.Embedding(len(unique_vegetarian_values) + 1, embedding_dimension)
      ])


      #vegan diet type
      tf.keras.layers.IntegerLookup(
          vocabulary = unique_vegan_values, mask_token=None),
      tf.keras.layers.Embedding(len(unique_vegan_values) + 1, embedding_dimension)
      # gluten-free diet type
      tf.keras.layers.IntegerLookup(
          vocabulary = unique_gf_values, mask_token=None),
      tf.keras.layers.Embedding(len(unique_gf_values) + 1, embedding_dimension)



  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)


____

In [ ]:
int_lookup = tf.keras.layers.IntegerLookup()
# int_lookup.adapt(recipe_ds.map(lambda x: x["vegetarian"]))
# int_lookup.adapt(recipe_ds.map(lambda x: x["vegan"]))
# int_lookup.adapt(recipe_ds.map(lambda x: x["GF"]))

KeyboardInterrupt: 

In [ ]:
# recipe_diet_embedding = tf.keras.layers.Embedding(
#     # Let's use the explicit vocabulary lookup.
#     input_dim=layer.vocabulary_size(),
#     output_dim=32
# )

In [ ]:
vege_embedding_model = tf.keras.Sequential([
  tf.keras.layers.Embedding(, 32)
])

#for timestamp in ratings.take(1).map(lambda x: x["timestamp"]).batch(1).as_numpy_iterator():
#  print(f"Timestamp embedding: {timestamp_embedding_model(timestamp)}.")



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
veg_recipe_embedding = tf.keras.layers.Embedding(
    # Let's use the explicit vocabulary lookup.
    input_dim=movie_title_lookup.vocab_size(),
    output_dim=32
)

